# Route4Me

Evaluation base line

- [Documentation](https://route4me.io/docs/)
- [Python SDK](https://github.com/route4me/route4me-python-sdk)

## Import

In [54]:
# Local modules
from modules import credentials, utilities

import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

import time
import ast

import requests
from IPython.display import JSON

import plotly.express as px
import plotly.graph_objects as go

import polyline
import folium

# Open Calgary API
from sodapy import Socrata

from route4me import Route4Me
from route4me.constants import (
    ALGORITHM_TYPE, DISTANCE_UNIT, TRAVEL_MODE, OPTIMIZE
)

## Load data

In [2]:
gdf = utilities.load_data()
gdf

,address,longitude,latitude,geometry
0,5532 DALWOOD WY NW,-114.162743,51.114109,POINT (-114.16274 51.11411)
1,321 MAGNOLIA HE SE,-113.914848,50.891335,POINT (-113.91485 50.89134)
2,165 CRANFIELD GR SE,-113.981177,50.888362,POINT (-113.98118 50.88836)
3,2 SAGE VALLEY ME NW,-114.150439,51.178209,POINT (-114.15044 51.17821)
4,714 12 ST SE,-114.031285,51.041928,POINT (-114.03129 51.04193)
...,...,...,...,...
1995,5715 36 ST SE,-113.981822,51.001674,POINT (-113.98182 51.00167)
1996,2028 URSENBACH RD NW,-114.136418,51.070260,POINT (-114.13642 51.07026)
1997,23 ROYAL OAK PZ NW,-114.217089,51.150980,POINT (-114.21709 51.15098)
1998,8045 48 ST SE,-113.964404,50.980330,POINT (-113.96440 50.98033)


In [6]:
gdf['address'].to_json()

TypeError: to_json() got an unexpected keyword argument 'drop_id'

In [3]:
r4m = Route4Me(credentials.R4M_API_Key)

In [8]:
addresses = [{'address': '455 S 4th St, Louisville, KY 40202'},
             {'address': '1604 PARKRIDGE PKWY, Louisville, KY, 40214'},
             {'address': '1407 MCCOY, Louisville, KY, 40215'},
             {'address': '4805 BELLEVUE AVE, Louisville, KY, 40215'},
             {'address': '730 CECIL AVENUE, Louisville, KY, 40211'}, ]
for address in addresses:
    print('Original Address: {0}'.format(address))
    geocode_error, address = r4m.address.fix_geocode(address)
    if geocode_error:
        print('Geocoding Error: {0}'.format(geocode_error))
    print('Geocoded Address: {0}'.format(address))

Original Address: {'address': '455 S 4th St, Louisville, KY 40202'}
Geocoded Address: {'address': '455 S 4th St, Louisville, KY 40202', 'lat': 38.251373, 'lng': -85.756971}
Original Address: {'address': '1604 PARKRIDGE PKWY, Louisville, KY, 40214'}
Geocoded Address: {'address': '1604 PARKRIDGE PKWY, Louisville, KY, 40214', 'lat': 38.141597, 'lng': -85.793871}
Original Address: {'address': '1407 MCCOY, Louisville, KY, 40215'}
Geocoded Address: {'address': '1407 MCCOY, Louisville, KY, 40215', 'lat': 38.202454, 'lng': -85.786505}
Original Address: {'address': '4805 BELLEVUE AVE, Louisville, KY, 40215'}
Geocoded Address: {'address': '4805 BELLEVUE AVE, Louisville, KY, 40215', 'lat': 38.178861, 'lng': -85.775014}
Original Address: {'address': '730 CECIL AVENUE, Louisville, KY, 40211'}
Geocoded Address: {'address': '730 CECIL AVENUE, Louisville, KY, 40211', 'lat': 38.248689, 'lng': -85.821064}


In [61]:
addresses = (gdf['address'][:10] + ' Calgary Alberta Canada').str.cat(sep='\n')

In [62]:
response = r4m.address.geocode(
    addresses=addresses,
    format='json'
).decode('utf-8')

In [63]:
pd.json_normalize(ast.literal_eval(response))

,address,lat,lng,type,normalized,confidence,original
0,"5532 Dalwood Way NW, Calgary, AB T3A 1S7, Canada",51.114209,-114.162902,premise,5532 dalwood wy nw calgary alberta canada\n321...,high,5532 DALWOOD WY NW Calgary Alberta Canada\n321...
1,"714 12 Street SE, Calgary, AB T2G 3H5, Canada",51.042366,-114.032847,street_address,5532 dalwood wy nw calgary alberta canada\n321...,high,5532 DALWOOD WY NW Calgary Alberta Canada\n321...
2,"321 Magnolia He SE, Calgary, AB T3M 2B6, Canada",50.891304,-113.914849,street_address,5532 dalwood wy nw calgary alberta canada\n321...,high,5532 DALWOOD WY NW Calgary Alberta Canada\n321...
3,"2 Sage Valley Dr NW, Calgary, AB T3R, Canada",51.182912,-114.147397,street_address,5532 dalwood wy nw calgary alberta canada\n321...,medium,5532 DALWOOD WY NW Calgary Alberta Canada\n321...


In [24]:
pd.json_normalize(response[0])

NotImplementedError: 

In [16]:
response

b'[{"address":"Dalwood, Axminster EX13, UK","lat":50.80003,"lng":-3.064961,"type":"locality, political","normalized":"5532 dalwood wy nw,321 magnolia he se,165 cranfield gr se,2 sage valley me nw,714 12 street se,103 deermont wy se,56 templegreen road ne,294 mahogany place se,59 sandpiper wy nw,1925 10a street sw","confidence":"medium","original":"5532 DALWOOD WY NW,321 MAGNOLIA HE SE,165 CRANFIELD GR SE,2 SAGE VALLEY ME NW,714 12 ST SE,103 DEERMONT WY SE,56 TEMPLEGREEN RD NE,294 MAHOGANY PL SE,59 SANDPIPER WY NW,1925 10A ST SW"},{"address":"Wyoming, USA","lat":43.075967,"lng":-107.290283,"type":"administrative_area_level_1, political","normalized":"5532 dalwood wy nw,321 magnolia he se,165 cranfield gr se,2 sage valley me nw,714 12 street se,103 deermont wy se,56 templegreen road ne,294 mahogany place se,59 sandpiper wy nw,1925 10a street sw","confidence":"medium","original":"5532 DALWOOD WY NW,321 MAGNOLIA HE SE,165 CRANFIELD GR SE,2 SAGE VALLEY ME NW,714 12 ST SE,103 DEERMONT WY SE,

In [9]:
for address in gdf['address'][:3]:
    print('Original Address: {0}'.format(address))
    geocode_error, address = r4m.address.fix_geocode(address)
    if geocode_error:
        print('Geocoding Error: {0}'.format(geocode_error))
    print('Geocoded Address: {0}'.format(address))

Original Address: 5532 DALWOOD WY NW


AttributeError: 'str' object has no attribute 'get'

In [3]:
optimization = r4m.optimization
address = r4m.address
optimization.algorithm_type(ALGORITHM_TYPE.TSP)
optimization.share_route(0)
optimization.store_route(0)
optimization.route_time(7 * 600)
optimization.rt(True)
optimization.route_max_duration(86400)
optimization.route_name('Single Driver Round Trip')
optimization.optimize(OPTIMIZE.TIME)
optimization.distance_unit(DISTANCE_UNIT.MI)
optimization.travel_mode(TRAVEL_MODE.DRIVING)
address.add_address(
    address='754 5th Ave New York, NY 10019',
    lat=40.7636197,
    lng=-73.9744388,
    alias='Bergdorf Goodman',
    is_depot=True,
    time=0
)
address.add_address(
    address='717 5th Ave New York, NY 10022',
    lat=40.7669692,
    lng=-73.9693864,
    alias='Giorgio Armani',
    time=60
)
address.add_address(
    address='888 Madison Ave New York, NY 10014',
    lat=40.7715154,
    lng=-73.9669241,
    alias='Ralph Lauren Women\'s and Home',
    time=60
)
address.add_address(
    address='1011 Madison Ave New York, NY 10075',
    lat=40.7772129,
    lng=-73.9669,
    alias='Yigal Azrou\u00ebl',
    time=60
)
address.add_address(
    address='440 Columbus Ave New York, NY 10024',
    lat=40.7808364,
    lng=-73.9732729,
    alias='Frank Stella Clothier',
    time=60
)
address.add_address(
    address='324 Columbus Ave #1 New York, NY 10023',
    lat=40.7803123,
    lng=-73.9793079,
    alias='Liana',
    time=60
)
address.add_address(
    address='110 W End Ave New York, NY 10023',
    lat=40.7753077,
    lng=-73.9861529,
    alias='Toga Bike Shop',
    time=60
)
address.add_address(
    address='555 W 57th St New York, NY 10019',
    lat=40.7718005,
    lng=-73.9897716,
    alias='BMW of Manhattan',
    time=60
)
address.add_address(
    address='57 W 57th St New York, NY 10019',
    lat=40.7558695,
    lng=-73.9862019,
    alias='Verizon Wireless',
    time=60
)

response = r4m.run_optimization()
print("Optimization Problem ID: {}".format(response.get("optimization_problem_id")))
for a in response.get("addresses", []):
    print('{} - {} - {}'.format(a.get("sequence_no"), a.get("alias"), a.get("address")))

Optimization Problem ID: F023857CD5F487810739CD6B025C2445
0 - Bergdorf Goodman - 754 5th Ave New York, NY 10019
1 - Giorgio Armani - 717 5th Ave New York, NY 10022
2 - Ralph Lauren Women's and Home - 888 Madison Ave New York, NY 10014
3 - Yigal Azrouël - 1011 Madison Ave New York, NY 10075
4 - Frank Stella Clothier - 440 Columbus Ave New York, NY 10024
5 - Liana - 324 Columbus Ave #1 New York, NY 10023
6 - Toga Bike Shop - 110 W End Ave New York, NY 10023
7 - BMW of Manhattan - 555 W 57th St New York, NY 10019
8 - Verizon Wireless - 57 W 57th St New York, NY 10019


In [ ]:
r4m.route